# Modelo Transformer usando TensorFlow

Los Transformers son una potente arquitectura de aprendizaje profundo utilizada para tareas de secuencia a secuencia, como la traducción de idiomas y la generación de texto. Se basan en un mecanismo llamado auto-atención para capturar las dependencias a través de secuencias de entrada de largo alcance. En este notebook, veremos cómo implementar un modelo Transformer desde cero utilizando TensorFlow.   

## Implementación

1. Importar las librerías necesarias

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, LayerNormalization
from tensorflow.keras.models import Model
import numpy as np

2025-03-24 11:47:41.046929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742816861.069142     886 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742816861.075573     886 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 11:47:41.097717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2. Codificación posicional   

La codificación posicional se añade a las incrustaciones de entrada para proporcionar información sobre la posición de los tokens en la secuencia. A diferencia de las RNN y las LSTM, los transformers no captan de forma inherente la naturaleza secuencial de los datos, por lo que las codificaciones posicionales son esenciales para inyectar esta información.

A continuación se muestra la función para calcular las codificaciones posicionales:

In [2]:
def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model) // 2)) / np.float32(d_model))
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)


- *Codificación posicional*: Esta función crea una codificación única para cada posición de la secuencia, que se añade a las incrustaciones de token.
- *Seno* y *coseno*: Las posiciones se codifican utilizando funciones seno y coseno con diferentes frecuencias para distinguir las posiciones.

3. Multi-Head Attention   

El mecanismo de atención multicabezal permite al modelo centrarse simultáneamente en distintas partes de la secuencia de entrada.    
Utiliza varias cabezas de atención para calcular distintas representaciones de la entrada.

In [3]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads
        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)


- `Multi-Head Attention`: Esta clase realiza la atención multicabezal dividiendo la entrada en múltiples cabezales, lo que permite al modelo centrarse en diferentes partes de la secuencia simultáneamente.
- `d_model` y `num_heads`: `d_model` es el tamaño de la incrustación y `num_heads` se refiere al número de cabezas de atención.
- Capas densas: Las transformaciones lineales de las consultas(queries), claves(keys) y valores(values) se crean mediante `wq`, `wk` y `wv`.

In [4]:
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])


- `split_heads`: Divide el tensor de entrada en varias cabezas. El tensor resultante tendrá la forma `(batch_size, num_heads, seq_len, depth)`.

In [5]:
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        
        attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        output = self.dense(attention)
        return output


- `call`: Este método realiza la operación de atención propiamente dicha. Primero calcula las consultas(queries), claves(keys) y valores(values) aplicando las capas densas correspondientes, las divide en cabezas y, a continuación, calcula la atención utilizando la función `scaled_dot_product_attention`.
- `scaled_dot_product_attention`: Calcula la atención usando la fórmula `scaled dot-product`.   

4. Scaled Dot-Product Attention   

`Scaled Dot Product Attention` es el mecanismo de atención central utilizado por el componente de atención multicabezal para calcular las puntuaciones de atención.    

Calcula el producto punto entre las consultas y las claves, escala el resultado, aplica una máscara (si es necesario) y, a continuación, calcula la suma ponderada de los valores en función de los pesos de atención.

In [6]:
    def scaled_dot_product_attention(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)
    
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights


***Clase completa:***

In [2]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads
        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        
        attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        output = self.dense(attention)
        return output
    
    def scaled_dot_product_attention(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)
    
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

5. Red Feed-Forward   

La red feed-forward posicional se utiliza para procesar cada posición de forma independiente:

In [3]:
class PositionwiseFeedforward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff):
        super(PositionwiseFeedforward, self).__init__()
        self.d_model = d_model
        self.dff = dff
        self.dense1 = Dense(dff, activation='relu')
        self.dense2 = Dense(d_model)
        
    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return x


- `PositionwiseFeedforward`: Esta clase aplica dos capas densas a cada posición de forma independiente. La primera capa transforma la entrada a una dimensión superior y la segunda la reduce de nuevo al tamaño original de `d_model`.
- `call`: Aplica las capas feed-forward secuencialmente a la entrada.

6. Bloque Transformer   

Este bloque combina la atención multicabezal y las redes feed-forward con capas de normalización y de dropout.

In [4]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedforward(d_model, dff)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, x, training, mask):
        attn_output = self.att(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2


- `TransformerBlock`: Este bloque combina atención multicabezal, capas feed-forward, de dropout y de normalización. El bloque es una unidad básica del modelo Transformer.
- `call`: La entrada pasa por la atención multicabezal, seguida de la eliminación y normalización de capas. A continuación, pasa a través de la red de alimentación hacia adelante con abandono y normalización adicionales.

7. Codificador   

El codificador esta formado por varias capas codificadoras. Convierte la secuencia de entrada en un conjunto de 'embeddings' enriquecidos con información posicional.   

- `Encoder`: El codificador se compone de una capa de 'embedding', una de codificación posicional, una de dropout y múltiples bloques `transformer`. Procesa la secuencia de entrada y genera una representación de la secuencia.
- `call`: La secuencia de entrada pasa por la capa de 'embedding', se añade la codificación posicional y, a continuación, atraviesa secuencialmente los bloques `transformer`.

In [5]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.dropout = Dropout(dropout_rate)
        self.enc_layers = [TransformerBlock(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
        return x


8. Decodificador    

El descodificador genera la secuencia de salida a partir de la representación codificada utilizando mecanismos para atender tanto a la salida del codificador como a los tokens generados previamente.   

- `call`:La secuencia de entrada pasa por el 'embedding' y la codificación posicional y, a continuación, por los bloques `transformer` del descodificador.

In [6]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.dropout = Dropout(dropout_rate)
        self.dec_layers = [TransformerBlock(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}
        x = self.embedding(x)
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.dec_layers[i](x, training, look_ahead_mask)
        return x, attention_weights


9. Modelo Transformer   

El modelo final combina el codificador y el descodificador y genera las predicciones finales.

In [7]:
class Transformer(Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super(Transformer, self).__init__()
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, dropout_rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, dropout_rate)
        self.final_layer = Dense(target_vocab_size)

    def build(self, input_shape):
        self.encoder.build(input_shape[0])  # Construir el encoder con la forma de la entrada
        self.decoder.build(input_shape[1])  # Construir el decoder con la forma del target
        self.final_layer.build(self.decoder.compute_output_shape(input_shape[1])) #Construir la capa final con la forma del output del decoder.
        super(Transformer, self).build(input_shape)

    def call(self, inputs, training, look_ahead_mask, padding_mask):
        inp, tar = inputs
        enc_output = self.encoder(inp, training, padding_mask)  # Asumiendo que padding_mask se usa en el encoder
        dec_output = self.decoder(tar, enc_output, training, look_ahead_mask, padding_mask) #Asumiendo que padding_mask se usa también en decoder.
        final_output = self.final_layer(dec_output)
        return final_output


10. Entrenamiento y pruebas.   

Preparación de los parámetros del modelo y ejecución de un primer entrenamiento con entradas de ejemplo:

In [8]:
# Definir los parámetros:
num_layers = 4
d_model = 128
num_heads = 8
dff = 512
input_vocab_size = 10000
target_vocab_size = 8000
maximum_position_encoding = 10000
dropout_rate = 0.1

In [9]:
transformer = Transformer(
                        num_layers= num_layers, 
                        d_model=d_model, 
                        num_heads=num_heads, 
                        dff=dff, 
                        input_vocab_size=input_vocab_size, 
                        target_vocab_size=target_vocab_size, 
                        maximum_position_encoding=maximum_position_encoding, 
                        dropout_rate=dropout_rate
                        )

NameError: name 'positional_encoding' is not defined

In [23]:
inputs = tf.random.uniform((64, 50), dtype=tf.int64, minval=0, maxval=input_vocab_size)
targets = tf.random.uniform((64, 50), dtype=tf.int64, minval=0, maxval=target_vocab_size)

look_ahead_mask = None
padding_mask = None

output = transformer((inputs, targets), training=True, look_ahead_mask=look_ahead_mask, padding_mask=padding_mask)
print(output.shape)

/usr/local/lib/python3.11/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'encoder_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'decoder_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


NotImplementedError: Layer Decoder does not have a `compute_output_shape` method implemented. Should implement `def compute_output_shape(self, input_shape)`.